**WITHOUT PIPELINE**

In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from sklearn.preprocessing import OneHotEncoder, MaxAbsScaler

In [5]:
from sklearn.tree import DecisionTreeClassifier

In [6]:
from sklearn.impute import SimpleImputer

In [7]:
import pickle

In [8]:
df= pd.read_csv('train.csv')

In [9]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [10]:
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True)

In [11]:
X = df.drop(columns=['Survived'])

In [12]:
y = df['Survived']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
df['Age'].value_counts()

,count
Age,
24.00,30
22.00,27
18.00,26
28.00,25
30.00,25
...,...
24.50,1
0.67,1
0.42,1


In [26]:
si_age = SimpleImputer()

In [42]:
si_embarked = SimpleImputer(strategy='most_frequent')

In [43]:
X_train_embarked = si_embarked.fit_transform(X_train[['Embarked']])

In [44]:
X_test_embarked = si_embarked.transform(X_test[['Embarked']])

In [45]:
ohe_sex = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

In [46]:
X_train_sex = ohe_sex.fit_transform(X_train[['Sex']])

In [47]:
X_test_sex = ohe_sex.transform(X_test[['Sex']])

In [48]:
ohe_embarked = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

In [49]:
X_train_embarked_ohe = ohe_embarked.fit_transform(X_train_embarked)

In [50]:
X_test_embarked_ohe = ohe_embarked.transform(X_test_embarked)

In [51]:
X_train_rem = X_train.drop(columns=['Sex', 'Age', 'Embarked'])

In [52]:
X_test_rem = X_test.drop(columns=['Sex', 'Age', 'Embarked'])

In [53]:
X_train_transformed = np.concatenate((X_train_rem, X_train_age, X_train_sex, X_train_embarked_ohe), axis=1)

In [54]:
X_test_transformed = np.concatenate((X_test_rem, X_test_age, X_test_sex, X_test_embarked_ohe), axis=1)

In [56]:
from sklearn.metrics import accuracy_score

In [57]:
clf = DecisionTreeClassifier()

In [58]:
clf.fit(X_train_transformed, y_train)

DecisionTreeClassifier()

In [59]:
y_pred = clf.predict(X_test_transformed)

In [60]:
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.7932960893854749


In [62]:
import os

if not os.path.exists('models'):
    os.makedirs('models')

pickle.dump(ohe_sex, open('models/ohe_sex.pkl', 'wb'))

In [65]:
pickle.dump(ohe_sex, open('models/ohe_sex.pkl', 'wb'))

In [63]:
pickle.dump(si_age, open('models/si_age.pkl', 'wb'))
pickle.dump(si_embarked, open('models/si_embarked.pkl', 'wb'))
pickle.dump(ohe_embarked, open('models/ohe_embarked.pkl', 'wb'))

In [64]:
pickle.dump(clf, open('models/clf.pkl', 'wb'))

**WITH PIPELINE**

In [67]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.tree import DecisionTreeClassifier
import pickle
df = pd.read_csv('train.csv')
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True)
X = df.drop(columns=['Survived'])
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [68]:
trf1 = ColumnTransformer([
    ('impute_age', SimpleImputer(), [2]),
    ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6])
], remainder='passthrough')


In [70]:
trf2 = ColumnTransformer([
    ('ohe_sex_embarked', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), [1, 6])
], remainder='passthrough')
trf3 = ColumnTransformer([
    ('scale', MinMaxScaler(), slice(0, 10))
])
trf4 = DecisionTreeClassifier()


In [71]:
pipe = Pipeline([
    ('trf1', trf1),
    ('trf2', trf2),
    ('trf3', trf3),
    ('trf4', trf4)
])


In [72]:
from sklearn import set_config
set_config(display='diagram')
pipe

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4', DecisionTreeClassifier())])

In [73]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4', DecisionTreeClassifier())])

In [74]:
y_pred_pipe = pipe.predict(X_test)

In [75]:
from sklearn.metrics import accuracy_score
print("Accuracy with Pipeline:", accuracy_score(y_test, y_pred_pipe))

Accuracy with Pipeline: 0.6256983240223464


In [76]:
import pickle

pickle.dump(pipe, open('models/pipeline.pkl', 'wb'))